## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-19-47-27 +0000,nyt,"Costco to Sell Ozempic and Wegovy, Weight-Loss...",https://www.nytimes.com/2025/10/05/health/ozem...
1,2025-10-05-19-47-00 +0000,wsj,Behind Chicago’s Tense Week: Protester Clashes...,https://www.wsj.com/us-news/behind-chicagos-te...
2,2025-10-05-19-39-46 +0000,nyt,"Macron Appoints French Cabinet, but Doubts Ove...",https://www.nytimes.com/2025/10/05/world/europ...
3,2025-10-05-19-39-33 +0000,nypost,West Chelsea’s Terminal Warehouse lands first ...,https://nypost.com/2025/10/05/business/west-ch...
4,2025-10-05-19-31-43 +0000,nypost,Driver fatally plows into granny crossing stre...,https://nypost.com/2025/10/05/us-news/driver-f...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
97,trump,33
84,gaza,12
9,chicago,10
147,deal,9
101,guard,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
70,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,119
172,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...,83
41,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...,79
19,2025-10-05-18-46-55 +0000,nyt,"Strong-Armed by Trump, Netanyahu Embraces Gaza...",https://www.nytimes.com/2025/10/05/world/middl...,71
13,2025-10-05-19-08-54 +0000,bbc,Trump authorises deployment of 300 National Gu...,https://www.bbc.com/news/articles/c2dnk0ee6pyo...,69


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
70,119,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
172,50,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...
154,41,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
153,32,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
41,30,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...
89,25,2025-10-05-12-00-00 +0000,nypost,Inside Howard Rubin’s perverted NYC penthouse ...,https://nypost.com/2025/10/05/us-news/inside-h...
108,24,2025-10-05-09-30-00 +0000,wsj,"As Laura Loomer, an influential loyalist to Pr...",https://www.wsj.com/politics/policy/laura-loom...
170,22,2025-10-04-20-42-26 +0000,latimes,The show must go on: Pacific Airshow attendees...,https://www.latimes.com/california/story/2025-...
162,22,2025-10-04-21-18-46 +0000,nypost,Former Syrian ruler Bashar al-Assad nearly die...,https://nypost.com/2025/10/04/world-news/ouste...
75,21,2025-10-05-14-17-52 +0000,wapo,Russian air attack across Ukraine damages ener...,https://www.washingtonpost.com/world/2025/10/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
